In [ ]:
pip install transformers datasets evaluate pandas

In [ ]:
from datasets import load_dataset

squad2 = load_dataset("squad_v2", split="train[:]")

In [ ]:
squad2 = squad2.train_test_split(test_size=0.2)

In [ ]:
squad2["train"][0]

{'id': '5acfb2a277cf76001a685948',
 'title': 'Royal_assent',
 'context': 'Royal assent is sometimes associated with elaborate ceremonies. In the United Kingdom, for instance, the sovereign may appear personally in the House of Lords or may appoint Lords Commissioners, who announce that royal assent has been granted at a ceremony held at the Palace of Westminster. However, royal assent is usually granted less ceremonially by letters patent. In other nations, such as Australia, the governor-general merely signs the bill. In Canada, the governor general may give assent either in person at a ceremony held in the Senate or by a written declaration notifying parliament of his or her agreement to the bill.',
 'question': 'What is not associated with elaborate ceremonies?',
 'answers': {'text': [], 'answer_start': []}}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        # Check if the question has no answerable context in SQuAD v2
        if len(answers[i]["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            answer = answers[i]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label it (0, 0)
            if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise, it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [ ]:
tokenized_squad2 = squad2.map(preprocess_function, batched=True, remove_columns=squad2["train"].column_names)

Map:   0%|          | 0/104255 [00:00<?, ? examples/s]

Map:   0%|          | 0/26064 [00:00<?, ? examples/s]

In [ ]:
tokenized_squad2["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 104255
})

In [ ]:
train_dataset = tokenized_squad2["train"]

In [ ]:
import tensorflow as tf
from keras import layers

def build_simple_qa_model(max_seq_length, vocab_size, embedding_dim, num_classes):
    # Input layers
    input_ids = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32, name="attention_mask")

    # Embedding layer
    embedding = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_ids)

    # Transformer layers (simplified)
    transformer_output = layers.LSTM(64, return_sequences=True)(embedding)

    # Question-Answering head
    qa_outputs = layers.Dense(num_classes, activation='softmax', name="qa_outputs")(transformer_output)

    # Model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=qa_outputs)

    return model

# Example usage
max_seq_length = 384
vocab_size = tokenizer.vocab_size  # Adjust based on your tokenizer's vocabulary size
embedding_dim = 768
num_classes = 2  # Number of classes for start and end positions

# Build the simplified model
simple_qa_model = build_simple_qa_model(max_seq_length, vocab_size, embedding_dim, num_classes)

# Compile the model (customize as needed)
simple_qa_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        metrics=["accuracy"])

# Verify the model summary
simple_qa_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 384)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 384, 768)             1920775   ['input_ids[0][0]']           
                                                          68                                      
                                                                                                  
 lstm (LSTM)                 (None, 384, 64)              213248    ['embedding[0][0]']           
                                                                                                  
 attention_mask (InputLayer  [(None, 384)]                0         []                        

In [ ]:
# Assuming your dataset is properly formatted with the required features
# Replace 'features' with the actual feature names in your dataset
features = ['input_ids', 'attention_mask']
labels = ['start_positions', 'end_positions']

# Assuming batch size of 4 for illustration
batch_size = 1024

# Train the model
simple_qa_model.fit(
    x={feature: train_dataset[feature] for feature in features},
    y={label: train_dataset[label] for label in labels},
    batch_size=batch_size,
    epochs=3,  # Adjust as needed
    validation_split=0.1  # Adjust validation split as needed
)